In [26]:
import pandas as pd
import numpy as np
data = pd.read_csv('/content/Twitter_Data.csv')
data["clean_text"].fillna("random", inplace=True)
data["category"].fillna(0.0, inplace=True)
data["category"] = data["category"].apply(np.int64)
data["category"] = data["category"].replace(1,2)
data["category"] = data["category"].replace(0,1)
data["category"] = data["category"].replace(-1,0)
print(data)

                                               clean_text  category
0       when modi promised “minimum government maximum...         0
1       talk all the nonsense and continue all the dra...         1
2       what did just say vote for modi  welcome bjp t...         2
3       asking his supporters prefix chowkidar their n...         2
4       answer who among these the most powerful world...         2
...                                                   ...       ...
162975  why these 456 crores paid neerav modi not reco...         0
162976  dear rss terrorist payal gawar what about modi...         0
162977  did you cover her interaction forum where she ...         1
162978  there big project came into india modi dream p...         1
162979  have you ever listen about like gurukul where ...         2

[162980 rows x 2 columns]


In [27]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
tweets = data["clean_text"].values.tolist()
classifiers = data["category"].values.tolist()

In [30]:
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, TFTrainer, TFTrainingArguments
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
tf.test.gpu_device_name()

'/device:GPU:0'

In [31]:
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base", num_labels=3)
print("------------------train and test splits --------------------------")
X_train, X_test, Y_train, Y_test = train_test_split(tweets, classifiers, test_size=0.2, random_state=0)

#
print("-------------------tokenizing------------------------------")
train_encodings = tokenizer(X_train, truncation=True, padding=True, return_tensors='np')
test_encodings = tokenizer(X_test, truncation=True, padding=True, return_tensors='np')
print(train_encodings)

------------------train and test splits --------------------------
-------------------tokenizing------------------------------
{'input_ids': array([[    0,  7877, 19639, ...,     1,     1,     1],
       [    0, 40808, 11134, ...,     1,     1,     1],
       [    0, 43956,   754, ...,     1,     1,     1],
       ...,
       [    0, 10932, 11134, ...,     1,     1,     1],
       [    0, 19827,   202, ...,     1,     1,     1],
       [    0,  8396,   340, ...,     1,     1,     1]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}


In [32]:
model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)

train_labels = np.array(Y_train)
test_labels = np.array(Y_test)
opt = tf.keras.optimizers.Adam(5e-5)  # Transformers like lower learning rates
model.compile(optimizer=opt, metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


----------------------------Training happens here -----------------------------

In [33]:
model.fit(dict(train_encodings), train_labels,epochs=5)
model.save('/content/customRobertaModel')

Epoch 1/5
4075/4075 [==============================] - 641s 147ms/step - loss: 0.2488 - sparse_categorical_accuracy: 0.9141
Epoch 2/5
4075/4075 [==============================] - 578s 142ms/step - loss: 0.1224 - sparse_categorical_accuracy: 0.9626
Epoch 3/5
4075/4075 [==============================] - 577s 142ms/step - loss: 0.0976 - sparse_categorical_accuracy: 0.9702
Epoch 4/5
4075/4075 [==============================] - 576s 141ms/step - loss: 0.0822 - sparse_categorical_accuracy: 0.9761
Epoch 5/5
4075/4075 [==============================] - 577s 142ms/step - loss: 0.0755 - sparse_categorical_accuracy: 0.9787


In [35]:
print("model is trained")

model is trained


In [37]:
!zip -r /content/customRobertaModel.zip /content/customRobertaModel/

  adding: content/customRobertaModel/ (stored 0%)
  adding: content/customRobertaModel/assets/ (stored 0%)
  adding: content/customRobertaModel/variables/ (stored 0%)
  adding: content/customRobertaModel/variables/variables.index (deflated 79%)
  adding: content/customRobertaModel/variables/variables.data-00000-of-00001 (deflated 19%)
  adding: content/customRobertaModel/keras_metadata.pb (deflated 96%)
  adding: content/customRobertaModel/saved_model.pb (deflated 93%)
  adding: content/customRobertaModel/fingerprint.pb (stored 0%)


In [38]:
from google.colab import files
files.download("/content/customRobertaModel.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>